In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import gmsh
import cv2
import scipy
import matplotlib as mpl
import plotly.graph_objs as go

from sklearn.neighbors import KNeighborsClassifier

In [ ]:
class IITM_Mesh:
    '''Class specific to IITM mesh'''
    def __init__(self, mesh_size) -> None:
        self.img = self.gmshIITM(mesh_size)
        self.M,self.K,self.S = self.Create_Matrices_2D()
           
    def gmshIITM(self,mesh_size):
        image = cv2.imread("IITM_MAP.png")
        img = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(img, 127, 255, 0)
        self.coordinate_label = np.array([[ coordinate_region(img[y][x]) for x in range(img.shape[1])] for y in range(img.shape[0])])
        contours, hierarchy = cv2.findContours(thresh, 3, 2)
        rdn = np.concatenate((contours[0][1:10],contours[0][41:55],contours[0][77:90],contours[0][116:160],contours[0][180:200],contours[0][234:266],contours[0][290:310],contours[0][327:392],contours[0][415:430],contours[0][452:519],contours[0][563:572],contours[0][631:634],contours[0][680:696],contours[0][703:705],contours[0][755:770],contours[0][840:846])) 
        gmsh.initialize()
        model = gmsh.model
        model.add("2D Model")
        # Scaling the contour points by factor of 100
        points = [model.geo.addPoint(point[0][0]/100, point[0][1]/100, 0, mesh_size) for point in np.flip(rdn)] 
        lines = [model.geo.addLine(points[i], points[(i+1)%len(points)]) for i in range(len(points))]
        curve_loop = model.geo.addCurveLoop(lines)
        model.geo.addPlaneSurface([curve_loop])
        model.geo.synchronize()
        model.mesh.generate(2)
        gmsh.write("GC.msh")
        gmsh.finalize()
        self.mshInfo()
        return img

    def mshInfo(self):
        'Change the location of the png'
        image = cv2.imread("IITM_MAP.png")
        img = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        # coordinate_label = np.array([[ coordinate_region(img[y][x]) for x in range(img.shape[1])] for y in range(img.shape[0])])
        ret, thresh = cv2.threshold(img, 127, 255, 0)
        contours, hierarchy = cv2.findContours(thresh, 3, 2)
        gc_contour = []
        area_of_Gajendra_circle = 1539
        for i in range(len(contours)):
            area = cv2.contourArea(contours[i])
            if area ==114:         #contour for Gajendra Circle 
                gc_contour.append(contours[i])
        scaling_factor_from_contourplot_reality = area_of_Gajendra_circle/(cv2.contourArea(gc_contour[0]))
        gmsh.initialize()
        #inorder to read the .msh file replace string "IITM_MAP_coarse.msh" with the address of IITM_MAP_coarse.msh
        'Change the location correspondingly'
        gmsh.open("GC.msh")
        # Extracting nodes from msh file
        nodeTags,coords,parametricCoord = gmsh.model.mesh.getNodes(-1)     
        coords = coords.reshape((-1,3))
        self.pred = self.classifyEachElement(coordinates=(coords[:,:2]*100))
        self.coords = coords[:,:2] * 100 * scaling_factor_from_contourplot_reality**.5
        # Extracting boundary mesh elements from msh file
        linearEleTags, linearNodesTags = gmsh.model.mesh.getElementsByType(1,-1)
        self.linearNodesTags = np.unique(linearNodesTags)
        # Extracting triangular mesh elements from msh file
        eleTags, triNodesTags = gmsh.model.mesh.getElementsByType(2,-1)
        self.triNodesTags = triNodesTags.reshape((-1,3))
        gc_contour_fine = np.array(gc_contour)*scaling_factor_from_contourplot_reality**.5
        # print(gc_contour_fine.shape)
        self.scaling_factor,self.x_gc,self.y_gc = (scaling_factor_from_contourplot_reality,np.mean(gc_contour_fine[0,:,0,0]),np.mean(gc_contour_fine[0,:,0,1]))

    def classifyEachElement(self,coordinates):
        X_train = np.array([ [y,x] for x in range(self.coordinate_label.shape[1]) for y in range(self.coordinate_label.shape[0]) ])
        y_train = np.array([ self.coordinate_label[y][x] for x in range(self.coordinate_label.shape[1]) for y in range(self.coordinate_label.shape[0])])
        knn = KNeighborsClassifier(n_neighbors = 100) 
        knn.fit(X_train, y_train) 
        return knn.predict(coordinates)

    def Create_Matrices_2D(self):
        triNodesTags = self.triNodesTags
        coords = self.coords
        # print(coords[0])
        n = len(coords)
        gaussPoints = 3
        sigma = 20
        mu = np.array([self.y_gc,self.x_gc])
        A = np.zeros((n,n))
        M = np.zeros((n,n))
        F = np.zeros((n,1))
        for ele in range(len(triNodesTags)):
            coordinates = coords[triNodesTags[ele]-1].T           #coordinates is dim * no. of points matrix
            values, counts = np.unique(k_values[self.pred[triNodesTags[ele]-1]], return_counts=True)
            ind = np.argmax(counts)
            k_element = values[ind]
            # k_element = 1
            values, counts = np.unique(rho_mul_Cp_values[self.pred[triNodesTags[ele]-1]], return_counts=True)
            ind = np.argmax(counts)
            rho_cp_element = values[ind]
            # rho_cp_element = 1
            for gpts in range(gaussPoints):
                pt = points_2D[gaussPoints][gpts]
                N = np.array([1-pt[0]-pt[1], pt[1], pt[0]]).reshape(1,-1)
                dN = np.array([[-1,0,1],[-1,1,0]]).T        #grad phi
                Jac = coordinates @ dN
                Jac_inv = np.linalg.inv(Jac)
                det_jab = np.linalg.det(Jac)
                m = (N.T@N)*det_jab*weights_2D[gaussPoints][gpts]*rho_cp_element
                # print((Jac_inv@dN).shape)
                a = (dN@Jac_inv)@((dN@Jac_inv).T)*det_jab*weights_2D[gaussPoints][gpts]*k_element
                X = (coordinates @ N.T).reshape(-1)
                # print([triNodesTags[ele]-1])
                f = N.T*Gaussian(X,sigma,mu)*det_jab*weights_2D[gaussPoints][gpts]
                M[np.ix_(triNodesTags[ele]-1,triNodesTags[ele]-1)] = M[np.ix_(triNodesTags[ele]-1,triNodesTags[ele]-1)] + m
                A[np.ix_(triNodesTags[ele]-1,triNodesTags[ele]-1)] = A[np.ix_(triNodesTags[ele]-1,triNodesTags[ele]-1)] + a
                F[triNodesTags[ele]-1] = F[triNodesTags[ele]-1] + f.reshape(-1,1)
        return M,A,F